<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#使用BERT初始化节点" data-toc-modified-id="使用BERT初始化节点-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>使用BERT初始化节点</a></span></li><li><span><a href="#NER方法" data-toc-modified-id="NER方法-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>NER方法</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#使用BERT进行NER" data-toc-modified-id="使用BERT进行NER-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>使用BERT进行NER</a></span></li><li><span><a href="#使用spacy-pipline-NER" data-toc-modified-id="使用spacy-pipline-NER-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>使用spacy pipline NER</a></span></li></ul></li></ul></li><li><span><a href="#检查" data-toc-modified-id="检查-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>检查</a></span></li><li><span><a href="#建立实体hyper-link" data-toc-modified-id="建立实体hyper-link-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>建立实体hyper-link</a></span></li><li><span><a href="#建立-adj,-labels" data-toc-modified-id="建立-adj,-labels-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>建立 adj, labels</a></span></li><li><span><a href="#第二阶段" data-toc-modified-id="第二阶段-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>第二阶段</a></span></li></ul></div>

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/folders/')
    !pip install transformers
    !pip install -U spacy[cuda100]
    !python -m spacy download en_core_web_lg
    # !wget -P /content/folders/My\ Drive/HotpotQA/ http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
    # json_train_path = '/content/folders/My Drive/HotpotQA/样例_hotpot_train_v1.1.json' # 5个例子
    json_train_path = '/content/folders/My Drive/HotpotQA/hotpot_train_v1.1.json'
except:
    json_train_path_mini = 'HotpotQA/样例_hotpot_train_v1.1.json'

In [ ]:
import json
import torch
import torch.nn as nn

In [16]:
with open(json_train_path, 'r', encoding='utf-8') as fp:
    json_train = json.load(fp)
    
json_train[0].keys()

dict_keys(['supporting_facts', 'level', 'question', 'context', 'answer', '_id', 'type'])

## 使用BERT初始化节点

使用`tokensizer`分词, 但是不能直接使用`model`的输出, 因为`model`的输入是句子对, 而且还有label. 应使用分词+`model`的`imput embedding matrix`.(`model`应是fine-turning之后的model)




In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForSequenceClassification.from_pretrained('roberta-base')

tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = RobertaForSequenceClassification.from_pretrained('roberta-large')

def tokensize_and_repr_in_BERT(raw_content,  tokenizer = tokenizer,\
                        model = model,\
                        max_length = 50,\
                        flatten = False):
    '''tokenizer和model需要对应.'''
    
    tokens_dict = tokenizer.encode(raw_content, add_special_tokens=False, max_length = max_length, \
                        pad_to_max_length = True, return_tensors = 'pt')

    embedding = model.get_input_embeddings()
    res = embedding(tokens_dict.squeeze())
    if flatten: res = res.reshape(1, -1)
    return res

## NER方法

#### 使用BERT进行NER

**暂时无法使用!** 需要获得char级别的span_start与end.



In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased-whole-word-masking")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

_ = model.eval()

In [ ]:
def find_NER_in_BERT(raw_content, model=model, tokenizer = tokenizer):
    label_list = [
        "O",       # Outside of a named entity
        "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
        "I-MISC",  # Miscellaneous entity
        "B-PER",   # Beginning of a person's name right after another person's name
        "I-PER",   # Person's name
        "B-ORG",   # Beginning of an organisation right after another organisation
        "I-ORG",   # Organisation
        "B-LOC",   # Beginning of a location right after another location
        "I-LOC"    # Location
        ]
    # Bit of a hack to get the tokens with the special tokens
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(raw_content)))
    inputs = tokenizer.encode(raw_content, return_tensors="pt")

    outputs = model(inputs)[0]
    predictions = torch.argmax(outputs, dim=2)

    res = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())]
    print(predictions)
    print(res)

    entities_list = []
    cursor_1 = 0
    cursor_2 = 0
    length = len(res)

    while cursor_1 < len(res):
        entities_dict = {}
        temp = []
        if res[cursor_1][1] == 'O': 
            cursor_1+=1
            continue
        
        entities_dict['type'] = res[cursor_1][1]
        entities_dict['span_start'] = cursor_1
        temp.append(res[cursor_1][0])
        cursor_2 = cursor_1 + 1
        while cursor_2 < len(res):
            if res[cursor_2][1] == 'O':
                cursor_1 = cursor_2 + 1
                break
            
            temp.append(res[cursor_2][0])
            cursor_2 += 1
        
        entities_dict['content'] = ' '.join(temp).replace(' ##', '')
        entities_dict['span_end'] = cursor_2
        entities_list.append(entities_dict)

    return entities_list

In [ ]:
s = "Radio City is India's first_private FM radio station, Radio City was started on 3 July 2001."
find_NER_in_BERT(s)

tensor([[0, 6, 6, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0]])
[('[CLS]', 'O'), ('Radio', 'I-ORG'), ('City', 'I-ORG'), ('is', 'O'), ('India', 'I-LOC'), ("'", 'O'), ('s', 'O'), ('first', 'O'), ('_', 'O'), ('private', 'O'), ('FM', 'O'), ('radio', 'O'), ('station', 'O'), (',', 'O'), ('Radio', 'I-ORG'), ('City', 'I-ORG'), ('was', 'O'), ('started', 'O'), ('on', 'O'), ('3', 'O'), ('July', 'O'), ('2001', 'O'), ('.', 'O'), ('[SEP]', 'O')]


[{'content': 'Radio City', 'span_end': 3, 'span_start': 1, 'type': 'I-ORG'},
 {'content': 'India', 'span_end': 5, 'span_start': 4, 'type': 'I-LOC'},
 {'content': 'Radio City', 'span_end': 16, 'span_start': 14, 'type': 'I-ORG'}]

#### 使用spacy pipline NER

In [ ]:
# !pip install -U spacy[cuda100]
# !python -m spacy download en_core_web_lg
# # 安装完需要重启kernel才能load

In [ ]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")

In [ ]:
def find_NER_in_nlp(raw_content, nlp = nlp, exclude_list = ['PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']):
    '''使用spacy进行NER.
    标注解释: https://spacy.io/api/annotation
    '''   
    entities_list = []
    for item in nlp(raw_content).ents:
        if item.label_ in exclude_list: continue
        entities_dict = {}
        entities_dict['type'] = item.label_
        entities_dict['span_start'] = item.start_char
        entities_dict['content'] = item.text
        entities_dict['span_end'] = item.end_char
        entities_list.append(entities_dict)
    # print(dir(item))
    return entities_list

In [19]:
s = "Radio City is India's first private FM radio station and was started on 3 July 2001."
find_NER_in_nlp(s)

[{'content': 'Radio City', 'span_end': 10, 'span_start': 0, 'type': 'GPE'},
 {'content': 'India', 'span_end': 19, 'span_start': 14, 'type': 'GPE'},
 {'content': '3 July 2001', 'span_end': 83, 'span_start': 72, 'type': 'DATE'}]

In [ ]:
class Node(object):
    '''Node class for graph'''
    
    def __init__(self, node_id, node_type, content_raw, content_NER_list, parent_id, content_features=None, is_support=False):
        self.node_id = node_id
        self.node_type = node_type
        self.content_raw = content_raw
        self.content_NER_list = content_NER_list
        self.content_features = content_features
        self.parent_id = parent_id
        
        # Q_node doesn't have.
        self.paragraph_id = -1
        self.start_in_paragraph = -1
        self.end_in_paragraph = -1
        
        self.is_support = is_support # 段落 句子 
        
    @classmethod
    def build(cls, node_id, node_type, content_raw, parent_id, tokensizer=None):
        # content_features = tokensize_and_repr_in_BERT(content_raw, flatten=True)
        content_NER_list = find_NER_in_nlp(content_raw) if node_type != 'Entity' else []
        # print(f'id:{node_id}\n{content_raw}\n{content_NER_list}\n')
        return cls(node_id, node_type, content_raw, content_NER_list, parent_id)
    
    def set_support(self):
        self.is_support = True
    
    def set_span_in_paragraph(self, para_id, start, end):
        self.paragraph_id = para_id
        self.start_in_paragraph = start
        self.end_in_paragraph = end
    
    def __str__(self):
        return f'Node: {self.node_type} {self.node_id}'
    
    def __repr__(self):
        return f'Node: {self.node_type} {self.node_id}'

    def get_NER_tuples_list(self):
        '''返回NER元组. e.g. [('ALLPE',id), ('DELL',id)]'''
        return [(i['content'], self.node_id) for i in self.content_NER_list]
        

In [21]:
node_id=11
node_type='ttt'
content_raw="Radio City is India's first private FM radio station and was started on 3 July 2001."

tn=Node.build(node_id,node_type,content_raw, -1)
tn

Node: ttt 11

In [22]:
tn.get_NER_tuples_list()

[('Radio City', 11), ('India', 11), ('3 July 2001', 11)]

In [ ]:
class Question_Paragraph(object):
    '''Q-P pair and label. for BERT and node init.'''
    def __init__(self, question, paragraph, ques_id, para_id, label:int):
        self.question = question
        self.paragraph = paragraph
        self.ques_id = ques_id
        self.para_id = para_id 
        self.label = label

        self.question_features = None # [N, dim]
        self.paragraph_features = None

    @classmethod
    def build(cls, question, paragraph, ques_id, para_id, label):
        return cls(question, paragraph, ques_id, para_id, label)

    # raw content
    def get_question(self):
        return self.question

    def get_paragraph(self):
        return self.paragraph

    def get_ques_para_pair(self):
        return [self.question, self.paragraph]

    def get_label(self):
        return self.label

    # features
    def build_features(self, LM_model):
        '''build features from LM models'''
        pass

    def get_question_features(self):
        return self.question_features

    def get_paragraph_features(self):
        return self.paragraph_features

    # other
    def __str__(self):
        return f'Q_P. p_id: {self.para_id}'
    
    def __repr__(self):
        return f'Q_P. p_id: {self.para_id}'


In [ ]:
import scipy.sparse as sp
import numpy as np

class Adjacency_sp(object):
    '''无重复稀疏邻接矩阵'''
    def __init__(self):
        self.v_i_j = []
        self.i_j_find_table = []

    def append(self, v, i, j):
        if not (i,j) in self.i_j_find_table:
          self.v_i_j.append([v,i,j])
          self.i_j_find_table.append((i,j))
    
    def to_dense(self, shape=(200, 200)):
        '''return numpy ndarray.'''
        np_adj = np.array(self.v_i_j)
        full_adj = sp.coo_matrix((np_adj[:, 0], (np_adj[:, 1], np_adj[:, 2])), shape=shape, dtype=np.float32).todense()
        full_adj = np.array(full_adj)
        return full_adj

    def to_dense_symmetric(self, shape=(200, 200)):
        np_adj = np.array(self.v_i_j)
        adj = sp.coo_matrix((np_adj[:, 0], (np_adj[:, 1], np_adj[:, 2])), shape=shape, dtype=np.float32)
        adj_symm = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj).todense()
        adj_symm = np.array(adj_symm)
        return adj_symm

    def __repr__(self):
        return f'Adjacency_sp has {len(self.v_i_j)} edges'
    def __str__(self):
        return f'Adjacency_sp has {len(self.v_i_j)} edges'
    def __len__(self):
        return len(self.v_i_j)

**4种点**

1. Question
2. Paragraph
3. Sentence
4. Entity

!cat /proc/cpuinfo

In [ ]:
# 返回Q-paragraph(for BERT); adj; node_list
edge_type_map = {
    'Q_P':1,
    'Q_E':2,
    'P_S':3,
    'S_S_hyper':4,
    'S_E':5,
    'P_P':6,
    'S_S':7,
}

def create_ques_info_dict():
    # 每个question item由5个元素组成
    ques_info_dict={}
    ques_info_dict['id'] = None
    ques_info_dict['node_list'] = None
    ques_info_dict['sp_adj'] = None
    ques_info_dict['ques_para_list'] = None
    return ques_info_dict

from collections import defaultdict
from tqdm import tqdm_notebook
def preprocessing(item_num = 2):
    resturn_list = []
    for item in tqdm_notebook(json_train[:item_num], \
                desc='processing json items'):
        ques_info_dict = create_ques_info_dict()

        supporting_facts = defaultdict(list)
        for s_fact in item['supporting_facts']:
            supporting_facts[s_fact[0]].append(s_fact[1])

        level = item['level']
        question = item['question']
        context = item['context']
        answer = item['answer']
        Q_id = item['_id']
        Q_type = item['type']

        ques_info_dict['id'] = Q_id

        node_list = []
        sp_adj = Adjacency_sp()
        index_cursor = 0
        
        Q_node_cursor = index_cursor
        Q_node = Node.build(Q_node_cursor, 'Question', question, -1)
        node_list.append(Q_node)

        ques_para_list = []
        
        for p_id, paragraph in enumerate(context):
            # Q-P content
            para_id = f'{Q_id}_{p_id}'
            paragraph_content = ''
            paragraph_offset = 0
            paragraph_label = 0

            title = paragraph[0]
            
            paragraph_content += title
            paragraph_offset += len(title)

            index_cursor += 1

            # 添加P-1和P
            try:
                if index_cursor != 0: sp_adj.append(edge_type_map['P_P'], P_node_cursor, index_cursor) 
            except (NameError): # 首次调用P_node_cursor会报错
                pass
            
            P_node_cursor = index_cursor
            P_node = Node.build(P_node_cursor, 'Paragraph', title, Q_node_cursor)
            P_node.set_span_in_paragraph(para_id, paragraph_offset - len(title), paragraph_offset)

            # 判断 support paragraph
            if title in supporting_facts.keys():
                paragraph_label = 1
                P_node.set_support()

            node_list.append(P_node)
            
            # 添加Q和P
            sp_adj.append(edge_type_map['Q_P'], Q_node_cursor, P_node_cursor)

    
            for s_index, sentence in enumerate(paragraph[1]):
                paragraph_content += sentence
                paragraph_offset += len(sentence)

                index_cursor += 1
                S_node_cursor = index_cursor
                S_node = Node.build(S_node_cursor, 'Sentence', sentence, P_node_cursor)
                S_node.set_span_in_paragraph(para_id, paragraph_offset - len(sentence), paragraph_offset)

                # 判断support fact
                if (paragraph_label == 1) and (s_index in supporting_facts[title]):
                    S_node.set_support()
                node_list.append(S_node)

                # 添加S之间边; P和S之间边
                if s_index != 0:
                    sp_adj.append(edge_type_map['S_S'], S_node_cursor - 1, S_node_cursor)
                sp_adj.append(edge_type_map['P_S'], P_node_cursor, S_node_cursor)

                # 添加S和E之间边
                for entities_dict in S_node.content_NER_list:
                    sentence_offset = paragraph_offset - len(sentence)

                    index_cursor += 1
                    E_node_cursor = index_cursor
                    E_node = Node.build(E_node_cursor, 'Entity', entities_dict['content'], S_node_cursor)
                    E_node.set_span_in_paragraph(para_id,\
                                    sentence_offset+entities_dict['span_start'],\
                                    sentence_offset+entities_dict['span_end'])

                    node_list.append(E_node)
                    sp_adj.append(edge_type_map['S_E'], S_node_cursor, E_node_cursor)

            # in paragraph loop.
            ques_para = Question_Paragraph.build(question, paragraph_content, Q_id, para_id, paragraph_label)
            ques_para_list.append(ques_para)
        
        ques_info_dict['node_list'] = node_list
        ques_info_dict['sp_adj'] = sp_adj
        ques_info_dict['ques_para_list'] = ques_para_list
        resturn_list.append(ques_info_dict)
    return resturn_list

In [31]:
test_res = preprocessing()

In [33]:
len(test_res)

2

## 检查

In [ ]:
# entity offset
E_nodes = [i for i in node_list if i.node_type == 'Entity']
print(f"len(E_nodes):{len(E_nodes)}")
test_start = int(E_nodes[77].start_in_paragraph)
test_end = int(E_nodes[77].end_in_paragraph)
print(E_nodes[77].paragraph_id)
print(E_nodes[77].content_raw)
r = [i for i in ques_para_list if i.para_id == E_nodes[77].paragraph_id][0].get_paragraph()
print(r)
print(f'offset: \n{r[test_start:test_end]}')

len(E_nodes):113
5a7a06935542990198eaf050_5
Thomas G. Spear
Arthur's MagazineArthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey's Lady's Book".
offset: 
Thomas G. Spear


In [ ]:
# sentence offset
S_nodes = [i for i in node_list if i.node_type == 'Sentence']
print(f"len(S_nodes):{len(S_nodes)}")
test_start = int(S_nodes[36].start_in_paragraph)
test_end = int(S_nodes[36].end_in_paragraph)
print(S_nodes[36].paragraph_id)
print(S_nodes[36].content_raw)
r = [i for i in ques_para_list if i.para_id == S_nodes[36].paragraph_id][0].get_paragraph()
print(r)
print(f'offset: \n{r[test_start:test_end]}')

len(S_nodes):46
5a7a06935542990198eaf050_7
 In 2011 the circulation of the magazine was 1,310,696 copies.
First for WomenFirst for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies.
offset: 
 In 2011 the circulation of the magazine was 1,310,696 copies.


In [ ]:
# NER识别情况
ner_nodes = []
for i in node_list:
    ner_nodes.extend(i.get_NER_tuples_list())

In [ ]:
# 段落节点的NER识别情况
P_nodes = [i for i in node_list if i.node_type=='Paragraph']
[i.content_NER_list for i in P_nodes]

[[{'content': 'Radio City', 'span_end': 10, 'span_start': 0, 'type': 'GPE'},
  {'content': 'Indian', 'span_end': 18, 'span_start': 12, 'type': 'NORP'}],
 [{'content': 'Albanian', 'span_end': 19, 'span_start': 11, 'type': 'NORP'}],
 [],
 [{'content': 'the Southern United States',
   'span_end': 46,
   'span_start': 20,
   'type': 'LOC'}],
 [{'content': 'First Arthur County Courthouse',
   'span_end': 30,
   'span_start': 0,
   'type': 'ORG'}],
 [{'content': "Arthur's Magazine",
   'span_end': 17,
   'span_start': 0,
   'type': 'ORG'}],
 [{'content': 'Ukrainian Hockey Championship',
   'span_end': 37,
   'span_start': 8,
   'type': 'ORG'}],
 [],
 [],
 [{'content': 'William Rast',
   'span_end': 12,
   'span_start': 0,
   'type': 'PERSON'}]]

In [ ]:
# 句子节点的识别情况
S_nodes = [i for i in node_list if i.node_type=='Sentence']
[i.content_NER_list for i in S_nodes]

[[{'content': 'Radio City', 'span_end': 10, 'span_start': 0, 'type': 'GPE'},
  {'content': 'India', 'span_end': 19, 'span_start': 14, 'type': 'GPE'},
  {'content': '3 July 2001',
   'span_end': 83,
   'span_start': 72,
   'type': 'DATE'}],
 [{'content': 'Mumbai', 'span_end': 74, 'span_start': 68, 'type': 'GPE'},
  {'content': '2004', 'span_end': 104, 'span_start': 100, 'type': 'DATE'},
  {'content': 'Bengaluru', 'span_end': 116, 'span_start': 107, 'type': 'ORG'},
  {'content': '2001', 'span_end': 139, 'span_start': 135, 'type': 'DATE'},
  {'content': 'Lucknow', 'span_end': 149, 'span_start': 142, 'type': 'GPE'},
  {'content': 'New Delhi', 'span_end': 163, 'span_start': 154, 'type': 'GPE'},
  {'content': '2003', 'span_end': 175, 'span_start': 171, 'type': 'DATE'}],
 [{'content': 'Hindi', 'span_end': 15, 'span_start': 10, 'type': 'LANGUAGE'},
  {'content': 'English',
   'span_end': 24,
   'span_start': 17,
   'type': 'LANGUAGE'}],
 [{'content': 'Hyderabad', 'span_end': 29, 'span_start': 

## 建立实体hyper-link

原文:

- 句子和段落节点相连.(通过相同实体)
- 问题节点和实体节点相连.


In [ ]:
# 连接Q节点和E节点.
Q_node = [i for i in node_list if i.node_type == 'Question']
assert Q_node != []
question = Q_node[0].content_raw
E_nodes = [i for i in node_list if i.node_type == 'Entity' and i.content_raw.replace(' ','') in question.replace(' ','')]
E_nodes

[Node: Entity 102]

In [ ]:
for i in E_nodes:
    sp_adj.append(edge_type_map['Q_E'], Q_node[0].node_id, i.node_id)

In [ ]:
# 连接S和P节点
# 注意!!原文没有使用entity linking系统, 而是直接使用Wikipedia提供的链接.
# 这里暂时使用字符串match.
S_node = [i for i in node_list if i.node_type == 'Sentence']
E_node = [i for i in node_list if i.node_type == 'Entity']
for E_n in E_node:
    entity = E_n.content_raw.replace(' ','')
    for S_n in S_node:
        if entity in S_n.content_raw.replace(' ',''):
            sp_adj.append(edge_type_map['S_S_hyper'], node_list[E_n.parent_id].node_id, node_list[S_n.parent_id].node_id)

## 建立 adj, labels

In [ ]:
labels = torch.stack([torch.tensor(float(node.is_support)) for node in node_list], dim=0)
print(labels)
print(labels.shape)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([170])


In [ ]:
sp_adj.to_dense()

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
sp_adj.to_dense_symmetric()

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 4., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# 会覆盖!
test_ajd = np.array([
    [9,1,2],
    [10,1,2],
    [3,1,3],
    [2,2,3],
    [3,3,4],
    ])
sp.coo_matrix((test_ajd[:,0], (test_ajd[:,1], test_ajd[:,2])), shape=(5,5)).todense()

matrix([[ 0,  0,  0,  0,  0],
        [ 0,  0, 19,  3,  0],
        [ 0,  0,  0,  2,  0],
        [ 0,  0,  0,  0,  3],
        [ 0,  0,  0,  0,  0]])

In [ ]:
# 第一阶段结束.
print(len(node_list))
print(len(symm_adj))
print(len(ques_para_list))

170
200
10


In [ ]:
# 保存为pickle
import 

SyntaxError: ignored

## 第二阶段

bert-fine-turning之后获得段落的表达. node初始化表达通过offset获得.

In [ ]:
ques_para_list[0].get_ques_para_pair()

In [ ]:
# features [N_nodes, hidden]
features = torch.stack([node.content_features.flatten() for node in node_list], dim=0)
features.shape